---
title: "Unsupervised Learning"
format:
    html: 
        code-fold: false
---

<br>
<br>

# Overview

### Part 1: Dimensionality Reduction

The objective of this section is to explore and demonstrate the effectiveness of PCA and t-SNE in reducing the dimensionality of complex data while preserving essential information and improving visualization.

1. **PCA (Principal Component Analysis):**
   - Apply PCA to your dataset.
   - Determine the optimal number of principal components.
   - Visualize the reduced-dimensional data.
   - Analyze and interpret the results.

2. **t-SNE (t-distributed Stochastic Neighbor Embedding):**
   - Implement t-SNE on the same dataset.
   - Experiment with different perplexity values.
   - Visualize the t-SNE output to reveal patterns and clusters.
   - Compare the results of t-SNE with those from PCA.

3. **Evaluation and Comparison:**
   - Evaluate the effectiveness of PCA and t-SNE in preserving data structure.
   - Compare the visualization capabilities of both techniques.
   - Discuss the trade-offs and scenarios where one technique may perform better than the other.

### Part 2: Clustering Methods

Apply clustering techniques (K-Means, DBSCAN, and Hierarchical clustering) to a selected dataset. The goal is to understand how each method works, compare their performance, and interpret the results.

1. **Clustering Methods:**
   - Apply **K-Means**, **DBSCAN**, and **Hierarchical clustering** to your dataset.
   - Write a technical summary for each method (2–4 paragraphs per method) explaining how it works, its purpose, and any model selection methods used (e.g., Elbow, Silhouette).

2. **Results Section:**
   - Discuss and visualize the results of each clustering analysis.
   - Compare the performance of different clustering methods, noting any insights gained from the analysis.
   - Visualize cluster patterns and how they relate (if at all) to existing labels in the dataset.
   - Use professional, labeled, and clear visualizations that support your discussion.

3. **Conclusion:**
   - Summarize the key findings and their real-world implications in a non-technical way. Focus on the most important results and how they could apply to practical situations.

# Code 

Provide the source code used for this section of the project here.

If you're using a package for code organization, you can import it at this point. However, make sure that the **actual workflow steps**—including data processing, analysis, and other key tasks—are conducted and clearly demonstrated on this page. The goal is to show the technical flow of your project, highlighting how the code is executed to achieve your results.

If relevant, link to additional documentation or external references that explain any complex components. This section should give readers a clear view of how the project is implemented from a technical perspective.

Remember, this page is a technical narrative, NOT just a notebook with a collection of code cells, include in-line Prose, to describe what is going on.